In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Question formulation 
### Assume you have USD100,000. You are looking at trading Coke and Pepsi. Assume cash earns 5% per year in the bank.

### On the 9th of October 2023, at the close of the market, assume you short sell USD100,000 shares of Coke stock, use the proceeds to buy USD100,000 shares of Pepsi.
### Simulate 1,000,000 daily paths of the pairs trade for the next 3 months (60 trading days). Be explicit with all the assumptions you make on the random walks you generate and how you think about their correlation and mean-reversion.
### If we enter the trade on the 9th what is the expected value of the trade on the 16th? What is the 5th and 95th percentile profit-loss? At what price differential would you enter this pairs trade, at what price would you exit? Why?
